In [30]:
pwd

'/home/nakyung/projects/BDAIFin/OFFLINE/client_card'

In [31]:
import importlib
import ServingColumns  

importlib.reload(ServingColumns)


<module 'ServingColumns' from '/home/nakyung/projects/BDAIFin/OFFLINE/client_card/ServingColumns.py'>

In [32]:
import sys
from pathlib import Path
import pandas as pd
from ServingColumns import (
    fit_artifacts, save_json,
    build_features, make_df_model
)

TRAIN_IN  = "../../DATA/dataset/train_stage2"
TEST_IN   = "../../DATA/dataset/test_stage2"

ART_PATH  = "../../DATA/artifacts/stage2_artifacts.json"

TRAIN_OUT = "../../DATA/dataset/TRAIN_stage2"
TEST_OUT  = "../../DATA/dataset/TEST_stage2"


In [33]:
df_train_raw = pd.read_parquet(TRAIN_IN)
df_test_raw  = pd.read_parquet(TEST_IN)

print("raw train:", df_train_raw.shape)
print("raw test :", df_test_raw.shape)

raw train: (5312525, 54)
raw test : (928867, 54)


In [34]:
# artifacts는 train에서만 fit
artifacts = fit_artifacts(df_train_raw, amt_q=0.9)

Path(ART_PATH).parent.mkdir(parents=True, exist_ok=True)
save_json(artifacts, ART_PATH)

print("saved artifacts:", ART_PATH)
print("thr_amt:", artifacts["thr_amt"], "| base_rate:", artifacts["base_rate"])
print("highrisk_mcc size:", len(artifacts["highrisk_mcc"]))


saved artifacts: ../../DATA/artifacts/stage2_artifacts.json
thr_amt: 4.707275390625 | base_rate: 0.0014471461310770302
highrisk_mcc size: 44


In [35]:
df_train_feat = build_features(df_train_raw, artifacts)
df_train_model = make_df_model(df_train_feat)

Path(TRAIN_OUT).parent.mkdir(parents=True, exist_ok=True)
df_train_model.to_parquet(TRAIN_OUT)

print("saved:", TRAIN_OUT, "| shape:", df_train_model.shape)

saved: ../../DATA/dataset/TRAIN_stage2 | shape: (5312525, 19)


In [36]:
df_test_feat = build_features(df_test_raw, artifacts)
df_test_model = make_df_model(df_test_feat)

Path(TEST_OUT).parent.mkdir(parents=True, exist_ok=True)
df_test_model.to_parquet(TEST_OUT)

print("saved:", TEST_OUT, "| shape:", df_test_model.shape)

saved: ../../DATA/dataset/TEST_stage2 | shape: (928867, 19)


In [37]:
train_cols = set(df_train_model.columns)
test_cols  = set(df_test_model.columns)

only_train = sorted(train_cols - test_cols)
only_test  = sorted(test_cols - train_cols)

print("only in train:", len(only_train))
print("only in test :", len(only_test))

if only_train[:20]:
    print("sample only_train:", only_train[:20])
if only_test[:20]:
    print("sample only_test:", only_test[:20])

assert train_cols == test_cols, "Train/Test columns mismatch!"
print("Train/Test columns match.")


only in train: 0
only in test : 0
Train/Test columns match.


In [38]:
train_cols

{'amount_deviation',
 'card_fraud_last3',
 'card_mcc_change_cnt_last5',
 'card_merchant_is_new',
 'card_velocity_spike_ratio',
 'client_fraud_last3',
 'client_mcc_repeat_cnt_last5',
 'client_merchant_is_new',
 'client_tx_1h_avg_prev',
 'current_age',
 'dev_x_mccnew',
 'fraud',
 'hour_cos',
 'hour_sin',
 'id',
 'log_yearly_income',
 'mccnew_x_velocity',
 'merchant_change_cnt_last5',
 'tx_hour'}